# Base Llama 3.1 vs PQC-tuned Llama Benchmark (Colab)

이 노트북은 Base Llama 3.1 모델과 PQC-tuned Llama 모델의 성능을 비교하는 벤치마크를 실행합니다.

**필요 사항:**
- Google Colab GPU (T4 이상 권장)
- Hugging Face 토큰 (Meta Llama 3.1 모델 접근 권한 필요)

**실행 시간:** 약 2-3시간 (79개 테스트 파일 × 2개 모델)

## 1. 환경 확인 및 설정

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# 현재 디렉토리 확인
import os
print(f"Current directory: {os.getcwd()}")

## 2. 저장소 클론 및 이동

In [ ]:
# GitHub 저장소 클론
!git clone https://github.com/BOB14th-project/AI--Benchmark.git
%cd AI--Benchmark

## 3. 필요한 패키지 설치

약 5-10분 소요됩니다.

In [ ]:
# 기본 패키지 설치 (8-bit quantization을 위해 bitsandbytes 추가)
!pip install -q transformers peft torch tqdm huggingface_hub accelerate bitsandbytes

In [ ]:
# Flash Attention 2 설치 (선택사항 - 속도 향상)
# 설치에 실패해도 벤치마크는 실행됩니다 (약간 느려질 수 있음)
!pip install -q flash-attn --no-build-isolation

## 4. Hugging Face 토큰 설정

**중요:** 아래 셀에서 `YOUR_HF_TOKEN_HERE`를 실제 Hugging Face 토큰으로 교체하세요.

토큰 발급 방법:
1. https://huggingface.co/settings/tokens 접속
2. "New token" 클릭
3. Read 권한으로 토큰 생성
4. Meta Llama 3.1 모델 접근 권한 신청: https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct

In [ ]:
import os

# 여기에 Hugging Face 토큰을 입력하세요
HF_TOKEN = "YOUR_HF_TOKEN_HERE"  # 예: "hf_xxxxxxxxxxxxx"

# 환경 변수로 설정
os.environ['HF_TOKEN'] = HF_TOKEN

print("✅ Hugging Face 토큰 설정 완료")

## 5. 테스트 파일 확인

In [ ]:
# 테스트 파일 및 Ground truth 확인
!echo "=== 테스트 파일 ==="
!ls data/test_files/assembly_binary/*.s | wc -l
!echo ""
!echo "=== Ground truth 파일 ==="
!ls data/ground_truth/assembly_binary/*.json | wc -l

## 6. 벤치마크 설정

테스트 개수를 조정할 수 있습니다:
- `TEST_LIMIT = None`: 전체 테스트 (79개, 약 2-3시간)
- `TEST_LIMIT = 10`: 빠른 테스트 (10개, 약 20-30분)
- `TEST_LIMIT = 3`: 매우 빠른 테스트 (3개, 약 5-10분)

In [ ]:
# 벤치마크 설정 수정 (선택사항)
# TEST_LIMIT 값을 변경하려면 아래 주석을 해제하고 실행하세요

# import re

# # 빠른 테스트 (10개 파일만)
# !sed -i 's/TEST_LIMIT = None/TEST_LIMIT = 10/' benchmark_base_vs_tuned_binary.py

# # 또는 전체 테스트 (기본값)
# # !sed -i 's/TEST_LIMIT = [0-9]*/TEST_LIMIT = None/' benchmark_base_vs_tuned_binary.py

# 현재 설정 확인
!grep "TEST_LIMIT =" benchmark_base_vs_tuned_binary.py

## 7. 벤치마크 실행

**경고:** 이 셀은 오랜 시간이 걸립니다!
- 전체 테스트: 약 2-3시간
- 10개 테스트: 약 20-30분

실행 중에 Colab 탭을 닫아도 되지만, 컴퓨터를 절전 모드로 두면 연결이 끊어질 수 있습니다.

In [ ]:
# 벤치마크 실행
!python3 benchmark_base_vs_tuned_binary.py

## 8. 결과 확인

In [ ]:
# 결과 파일 확인
!ls -lh results/base_vs_tuned_binary_*.json

In [ ]:
# 최신 결과 파일 내용 확인
import json
import glob
from pathlib import Path

# 가장 최근 결과 파일 찾기
result_files = sorted(glob.glob("results/base_vs_tuned_binary_*.json"), reverse=True)

if result_files:
    latest_result = result_files[0]
    print(f"📊 최신 결과 파일: {latest_result}\n")
    
    with open(latest_result, 'r') as f:
        data = json.load(f)
    
    # 요약 정보 출력
    print("=" * 80)
    print("벤치마크 정보")
    print("=" * 80)
    print(f"실행 시간: {data['benchmark_info']['timestamp']}")
    print(f"총 테스트 수: {data['benchmark_info']['total_tests']}")
    print(f"Base 모델: {data['benchmark_info']['base_model']}")
    print(f"Tuned 모델: {data['benchmark_info']['tuned_model']}")
    
    # Base 모델 메트릭 계산
    base_results = [r for r in data['base_results'] if 'error' not in r]
    if base_results:
        base_tp = sum(r['true_positives'] for r in base_results)
        base_fp = sum(r['false_positives'] for r in base_results)
        base_fn = sum(r['false_negatives'] for r in base_results)
        base_precision = base_tp / (base_tp + base_fp) if (base_tp + base_fp) > 0 else 0
        base_recall = base_tp / (base_tp + base_fn) if (base_tp + base_fn) > 0 else 0
        base_f1 = 2 * base_precision * base_recall / (base_precision + base_recall) if (base_precision + base_recall) > 0 else 0
        
        print("\n" + "=" * 80)
        print("Base Llama 3.1 결과")
        print("=" * 80)
        print(f"Precision: {base_precision:.3f}")
        print(f"Recall: {base_recall:.3f}")
        print(f"F1-Score: {base_f1:.3f}")
        print(f"TP: {base_tp}, FP: {base_fp}, FN: {base_fn}")
    
    # Tuned 모델 메트릭 계산
    tuned_results = [r for r in data['tuned_results'] if 'error' not in r]
    if tuned_results:
        tuned_tp = sum(r['true_positives'] for r in tuned_results)
        tuned_fp = sum(r['false_positives'] for r in tuned_results)
        tuned_fn = sum(r['false_negatives'] for r in tuned_results)
        tuned_precision = tuned_tp / (tuned_tp + tuned_fp) if (tuned_tp + tuned_fp) > 0 else 0
        tuned_recall = tuned_tp / (tuned_tp + tuned_fn) if (tuned_tp + tuned_fn) > 0 else 0
        tuned_f1 = 2 * tuned_precision * tuned_recall / (tuned_precision + tuned_recall) if (tuned_precision + tuned_recall) > 0 else 0
        
        print("\n" + "=" * 80)
        print("PQC-tuned Llama 결과")
        print("=" * 80)
        print(f"Precision: {tuned_precision:.3f}")
        print(f"Recall: {tuned_recall:.3f}")
        print(f"F1-Score: {tuned_f1:.3f}")
        print(f"TP: {tuned_tp}, FP: {tuned_fp}, FN: {tuned_fn}")
        
        # 개선율 계산
        if base_f1 > 0:
            improvement = ((tuned_f1 - base_f1) / base_f1) * 100
            print("\n" + "=" * 80)
            print("개선율")
            print("=" * 80)
            print(f"F1-Score 개선: {improvement:+.2f}%")
else:
    print("❌ 결과 파일을 찾을 수 없습니다.")

## 9. 결과 다운로드

결과 파일을 로컬 컴퓨터로 다운로드합니다.

In [ ]:
from google.colab import files

# 가장 최근 결과 파일 다운로드
result_files = sorted(glob.glob("results/base_vs_tuned_binary_*.json"), reverse=True)

if result_files:
    latest_result = result_files[0]
    print(f"📥 다운로드 중: {latest_result}")
    files.download(latest_result)
    print("✅ 다운로드 완료!")
else:
    print("❌ 다운로드할 결과 파일이 없습니다.")

## 10. 추가 분석 (선택사항)

개별 테스트 케이스별 결과를 확인할 수 있습니다.

In [ ]:
# 개별 테스트 결과 확인
import pandas as pd

if result_files:
    with open(latest_result, 'r') as f:
        data = json.load(f)
    
    # Base 결과를 DataFrame으로 변환
    base_df = pd.DataFrame(data['base_results'])
    tuned_df = pd.DataFrame(data['tuned_results'])
    
    print("=== Base Model - 상위 10개 결과 ===")
    if 'test_id' in base_df.columns:
        display(base_df[['test_id', 'true_positives', 'false_positives', 'false_negatives', 'response_time']].head(10))
    
    print("\n=== Tuned Model - 상위 10개 결과 ===")
    if 'test_id' in tuned_df.columns:
        display(tuned_df[['test_id', 'true_positives', 'false_positives', 'false_negatives', 'response_time']].head(10))

## 문제 해결

### GPU 메모리 부족
- TEST_LIMIT을 더 작은 값으로 설정하세요
- 런타임 재시작 후 다시 실행하세요

### Hugging Face 토큰 오류
- 토큰이 올바른지 확인하세요
- Meta Llama 3.1 모델 접근 권한을 신청했는지 확인하세요

### 연결 끊김
- Colab Pro를 사용하면 더 긴 실행 시간을 보장받을 수 있습니다
- 중간 결과는 자동으로 저장되지 않으므로, 처음부터 다시 실행해야 합니다